<a href="https://colab.research.google.com/github/anujR1994/Data-Science-Projects/blob/main/Anuj_Rastogi_Graded_NN_Microproject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [96]:
! pip install pyforest

In [97]:
import pyforest


In [98]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [99]:
# Load data

df = pd.read_csv('/content/Disastertweets.csv')

<IPython.core.display.Javascript object>

In [100]:
df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [101]:
df.target.value_counts()

target
0    4342
1    3271
Name: count, dtype: int64

In [102]:
df.keyword.isnull().sum().sum()

61

In [103]:
df.location.isnull().sum().sum()

2533


**Text pre-processing steps are as follows:**

*   Tokenize the words
*   Remove the stop words from the received tokens
*   Lemmatize the words removing pos='v'
*   Define pipeline with the previous steps as functions
*   pl = [str.lower,tokenize,remove_stop,lemmatize1]
*   Create a new column: tokens containing transformed tokens
*   Create a new column named 'cleaned text' by joining the tokens again
*   Perform train test split with 'cleanedtext' and 'target' column
*   Import count vectorizer to perform bag of words and define ngram range (1,2)
*   Create a dataframe.
*   Import the respective models as per business problem.
*   Perform hyper parameter tuning to improve model performance.











In [104]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


In [105]:
# Filling null values
df['keyword'].fillna('no_keyword',inplace=True)
df['location'].fillna('no_location',inplace=True)

In [106]:
df.head()

,id,keyword,location,text,target
0,1,no_keyword,no_location,Our Deeds are the Reason of this #earthquake M...,1
1,4,no_keyword,no_location,Forest fire near La Ronge Sask. Canada,1
2,5,no_keyword,no_location,All residents asked to 'shelter in place' are ...,1
3,6,no_keyword,no_location,"13,000 people receive #wildfires evacuation or...",1
4,7,no_keyword,no_location,Just got sent this photo from Ruby #Alaska as ...,1


In [107]:
import re # regular expressions
import nltk # natural language toolkit
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [108]:
stop1 = stopwords.words('english')

In [109]:
# Creating function to tokenize the text input
def tokenize(text):
  return re.findall(r'\w+',text)

In [110]:
# Creating function to remove stopwords from the received tokens
def remove_stop(tokens):
  return [t for t in tokens if t.lower() not in stopwords.words('english')]

In [111]:
# Creating a function to lemmatize the tokens
def lemmatize1(tokens):
  lemmatizer = WordNetLemmatizer()
  return [lemmatizer.lemmatize(t,pos='v') for t in tokens]

In [112]:
# Create a pipeline to perfrom the above pre-processing with the text input
def prepare(text, pipeline):
  tokens = text
  for transform in pipeline:
    tokens = transform(tokens)
  return tokens

In [113]:
# Defining the pipeline 'pl'
pl = [str.lower,tokenize,remove_stop,lemmatize1]


In [114]:
# Creating a new column 'tokens' post applying 'pipeline transformation' to the text input
df['tokens'] = df['text'].apply(prepare, pipeline = pl)

In [115]:
# Creating a 'cleanedtext' column by joining the tokens again
df['cleanedtext'] = df['tokens'].apply(lambda x: ' '.join(x))

In [116]:
# Reading the new dataframe post processing
df.head()

,id,keyword,location,text,target,tokens,cleanedtext
0,1,no_keyword,no_location,Our Deeds are the Reason of this #earthquake M...,1,"[deeds, reason, earthquake, may, allah, forgiv...",deeds reason earthquake may allah forgive us
1,4,no_keyword,no_location,Forest fire near La Ronge Sask. Canada,1,"[forest, fire, near, la, ronge, sask, canada]",forest fire near la ronge sask canada
2,5,no_keyword,no_location,All residents asked to 'shelter in place' are ...,1,"[residents, ask, shelter, place, notify, offic...",residents ask shelter place notify officer eva...
3,6,no_keyword,no_location,"13,000 people receive #wildfires evacuation or...",1,"[13, 000, people, receive, wildfires, evacuati...",13 000 people receive wildfires evacuation ord...
4,7,no_keyword,no_location,Just got sent this photo from Ruby #Alaska as ...,1,"[get, send, photo, ruby, alaska, smoke, wildfi...",get send photo ruby alaska smoke wildfires pou...


In [117]:
# Target distribution
df.target.value_counts()

target
0    4342
1    3271
Name: count, dtype: int64

In [118]:
# Importing train text split
from sklearn.model_selection import train_test_split

In [119]:
xtrain,xtest,ytrain,ytest = train_test_split(df['cleanedtext'],df['target'],test_size=0.2,stratify=df['target'])

**Data Encoding using Bag of words BOW or TF_IDF**

In [120]:
# Importing CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer

In [121]:
# Initializing the bagofwords BOW
BOW = CountVectorizer(ngram_range=(1,2), stop_words='english')

In [122]:
Xtrain_bow = BOW.fit_transform(xtrain)

Xtest_bow = BOW.transform(xtest)

In [123]:
Xtrain_bow.shape

(6090, 57388)

In [124]:
Xtest_bow.shape

(1523, 57388)

In [125]:
# Creating dense matrix
Xtrain_bow.todense()


matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])

In [126]:
# Printing features post encoding
BOW.get_feature_names_out()

array(['00', '00 11', '00 18', ..., 'ûótech business', 'ûówe',
       'ûówe work'], dtype=object)

In [128]:
# Creating a dataframe with the dense matrix
df_bow = pd.DataFrame(Xtrain_bow.todense(),columns=BOW.get_feature_names_out())

df_bow

<IPython.core.display.Javascript object>

,00,00 11,00 18,00 bestseller,00 ep,00 hiroshima,00 http,00 nnw,00 pm,00 rockingham,...,ûó wallybaiter,ûó ûªm,ûókody,ûókody vine,ûónegligence,ûónegligence fireworks,ûótech,ûótech business,ûówe,ûówe work
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6085,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6086,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6087,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6088,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


**Model Building with the help of above dataframe**

In [129]:
# Importing Logistic Regression
from sklearn.linear_model import LogisticRegression

In [131]:
# Initializing model object
lr = LogisticRegression()

In [133]:
# Training the model on training data
lr.fit(Xtrain_bow,ytrain)

LogisticRegression()

In [134]:
# Getting prediction on test or unseen data
y_pred = lr.predict(Xtest_bow)

**Model Performance**

In [135]:
# Importing Accuracy
from sklearn.metrics import accuracy_score

In [136]:
# Testing accuracy
accuracy_score(ytest,y_pred)

0.7905449770190414